In [4]:
import pandas as pd
import numpy as np
from collections import Counter
import dataio
pd.set_option('max.columns', None)
pd.set_option('max.rows',None)

# profile = dataio.SnowflakeProfile(name='kkurek@uplift.com')
# dm_snowflake = dataio.SnowflakeDataManager(profile)

# User name: kkurek@uplift.com
# Private key path: /Users/kevinkurek/Desktop/rsa_key.p8
# Role: T_LENDING_FRAUD_PII
# Database: PRODUCTION_DB or ANALYTICS_DB
# Warehouse: ADHOC_WH
# Key encryption password:········

# Dan pre = previous transaction logic & if; checking previous payment token
# df_ = dm_snowflake.query("""
# select txn_timestamp, application_id, hour, dayofweek, transaction_provider,
#         transaction_status, txn_duration, provider_response_code,
#         type, bank, rsubtype, transaction_amount, financed_amount, term, fico,
#         payment_index, destination_account,
#         if_mannual_pay, if_change_card, if_change_processor,
#         direction_change_processor, monthly_vintage, isocountry
        
# from ANALYTICS_DB.DEV.DS_FCS_CARD_TXN_V
# where txn_timestamp > '2019-01-01'
# order by txn_timestamp""")
# # print(df_.shape)
# # display(df_.head())

# Clean for baseline MAB
df_orig = pd.read_csv("static_processor_data_20201002.csv", index_col=0)
print(f"Original data shape: {df_orig.shape}")

# Lets focus on the USA for now, and only a few primary columns to test if a MAB learns along with some EDA
df_orig = df_orig[df_orig['isocountry']=='UNITED STATES']

# Only rows with not null bank value
df_orig = df_orig[df_orig['bank'].notna()]

# Define df_ which means no retries
df_ = df_orig[df_orig['if_retry']==0]
print(f"No Retry data shape: {df_.shape}")
display(df_.head(3))

# Define df_retry into retries
df_retry = df_orig[df_orig['if_retry']!=0]
print(f"Yes Retry data shape: {df_retry.shape}")
display(df_retry.head(3))

def prep_data(df_):

    # Define Reward Column
    df_['reward'] = np.where(df_['transaction_status']=='SUCCEEDED', 1, 0)
    print(Counter(df_['reward']), Counter(df_['transaction_provider']))
    # display(df_.head())

    # How many arms should we have; 3 * 24 * 7 = 504
    print(len(df_['transaction_provider'].unique()), len(df_['hour'].unique()), len(df_['dayofweek'].unique()))

    # Convert processor * hour * dayofweek to categorical numeric arms.
    df_['arms'] = df_.groupby(['transaction_provider','hour','dayofweek']).ngroup()

    # How many arms did we actually get: 502. Missing 2 combinations somewhere; likely never processed at that combo.
    print(f"Number of unique Arms: {len(df_['arms'].unique())}")

    # Decoding of how each processor/hour/dayofweek maps to specific arm
    decode_df = df_.drop_duplicates(['arms'], keep='first').sort_values(by='arms').reset_index(drop=True)
    decode_df = decode_df[['transaction_provider','hour','dayofweek', 'arms']]
    display(decode_df.head())

    df = df_.drop(['transaction_status', 'provider_response_code', 'rsubtype', 'isocountry', 'if_retry'], axis=1)
    print(df.shape)
    
    return df
    
df = prep_data(df_)
display(df.head(3))
df_retry = prep_data(df_retry)
display(df_retry.head(3))

df.to_csv("VW_CMAB_data_NoRetry.csv")
df_retry.to_csv("VW_CMAB_data_YesRetry.csv")

/Users/kevinkurek/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (12,13,25,27) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/kevinkurek/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Original data shape: (2488733, 27)
No Retry data shape: (1871795, 27)


,txn_timestamp,loan_id,application_id,hour,dayofweek,transaction_provider,pre_txn_provider,transaction_status,txn_duration,provider_response_code,pre_response_code,type,bank,rsubtype,transaction_amount,financed_amount,term,fico,payment_index,destination_account,if_retry,if_mannual_pay,if_change_card,if_change_processor,direction_change_processor,monthly_vintage,isocountry
73164,2019-04-05 23:33:50.463,34351289,C43516437946,23,Fri,Stripe,Stripe,SUCCEEDED,10.0,succeeded,succeeded,DEBIT,USAA,card-sale,1757.36,2987.57,11,632.0,6.0,fortress1,0,1,1,0,NaN,2018-10,UNITED STATES
73165,2019-04-05 23:34:31.855,32461270,C24623226959,23,Fri,Stripe,Stripe,SUCCEEDED,0.0,succeeded,do_not_honor,DEBIT,USAA SAVINGS BANK,card-sale,190.50,1882.19,11,717.0,4.0,fortress1,0,1,1,0,NaN,2018-6,UNITED STATES
73166,2019-04-05 23:40:53.843,40217938,D02103129260,23,Fri,Stripe,NaN,SUCCEEDED,NaN,succeeded,NaN,DEBIT,BMO HARRIS BANK N.A.,card-sale,270.51,322.88,11,718.0,1.0,fortress1,0,1,0,0,NaN,2019-1,UNITED STATES


Yes Retry data shape: (433162, 27)


,txn_timestamp,loan_id,application_id,hour,dayofweek,transaction_provider,pre_txn_provider,transaction_status,txn_duration,provider_response_code,pre_response_code,type,bank,rsubtype,transaction_amount,financed_amount,term,fico,payment_index,destination_account,if_retry,if_mannual_pay,if_change_card,if_change_processor,direction_change_processor,monthly_vintage,isocountry
73206,2019-04-06 01:04:42.306,33852310,C38513053546,1,Sat,Stripe,Stripe,ERRORED,0.0,insufficient_funds,do_not_honor,CREDIT,CREDIT ONE BANK N.A.,card-sale,106.24,1001.52,11,591.0,4.0,fortress1,1,1,1,0,NaN,2018-9,UNITED STATES
73208,2019-04-06 01:06:45.940,33852310,C38513053546,1,Sat,Stripe,Stripe,SUCCEEDED,0.0,succeeded,insufficient_funds,DEBIT,CAPITAL ONE BANK (USA) N.A.,card-sale,111.24,1001.52,11,591.0,4.0,fortress1,1,1,1,0,NaN,2018-9,UNITED STATES
73210,2019-04-06 01:11:37.658,34936973,C49320092522,1,Sat,Stripe,Stripe,ERRORED,1.0,succeeded,succeeded,DEBIT,PNC BANK N.A.,card-sale,171.10,1615.96,11,603.0,3.0,fortress1,1,1,0,0,NaN,2018-12,UNITED STATES


...

Number of unique Arms: 425


/Users/kevinkurek/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,transaction_provider,hour,dayofweek,arms
0,Chase,0,Mon,0
1,Chase,0,Sun,1
2,Chase,1,Mon,2
3,Chase,1,Sun,3
4,Chase,1,Tue,4


(433162, 24)


,txn_timestamp,loan_id,application_id,hour,dayofweek,transaction_provider,pre_txn_provider,txn_duration,pre_response_code,type,bank,transaction_amount,financed_amount,term,fico,payment_index,destination_account,if_mannual_pay,if_change_card,if_change_processor,direction_change_processor,monthly_vintage,reward,arms
73206,2019-04-06 01:04:42.306,33852310,C38513053546,1,Sat,Stripe,Stripe,0.0,do_not_honor,CREDIT,CREDIT ONE BANK N.A.,106.24,1001.52,11,591.0,4.0,fortress1,1,1,0,NaN,2018-9,0,98
73208,2019-04-06 01:06:45.940,33852310,C38513053546,1,Sat,Stripe,Stripe,0.0,insufficient_funds,DEBIT,CAPITAL ONE BANK (USA) N.A.,111.24,1001.52,11,591.0,4.0,fortress1,1,1,0,NaN,2018-9,1,98
73210,2019-04-06 01:11:37.658,34936973,C49320092522,1,Sat,Stripe,Stripe,1.0,succeeded,DEBIT,PNC BANK N.A.,171.10,1615.96,11,603.0,3.0,fortress1,1,0,0,NaN,2018-12,0,98


In [3]:
# Have to install vowpal wabbit using the wheels in this link
# https://github.com/VowpalWabbit/vowpal_wabbit/issues/1764#issuecomment-701379336

import pandas as pd
import sklearn as sk
import numpy as np
pd.set_option('max.columns', None)
pd.set_option('max.rows',None)

# Read in data
df = pd.read_csv('CMAB_data.csv', index_col=0)
df.head(3)

# Find historical probability of each arm
df_pct = pd.DataFrame(df.groupby(['arms', 'reward'])['reward'].count()).rename(columns={'reward':'probability'})
df_pct = df_pct.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))
df_pct = df_pct.reset_index()

# Merge original df with probability of each arm
df_merged = df.merge(df_pct, on=['arms', 'reward'], how='inner')

# Move action(arms), cost(reward), probability calculation to front as vowpal wabbit expects
cols_to_move = ['arms', 'reward', 'probability']
df_merged = df_merged[ cols_to_move + [ col for col in df_merged.columns if col not in cols_to_move ] ]

# Adding sequential index as vowpal wabbit expects
df_merged['index'] = range(1, len(df_merged) + 1)
df_merged = df_merged.set_index("index")

# Dropping excess features already represented in data elsewhere
df_merged = df_merged.drop(['txn_timestamp', 'application_id'], axis=1)

# display(df_merged[df_merged['arms']==310]) Yes probabilities are p & 1-p
df_merged.head(3)

VW optimizes to minimize **cost which is negative of reward**. Therefore, we will always pass negative of reward as cost to VW.

In [39]:
df_merged['reward'] = df_merged['reward'] * -1
df_merged.head(3)

,arms,reward,probability,hour,dayofweek,transaction_provider,txn_duration,type,bank,transaction_amount,financed_amount,term,fico,payment_index,destination_account,if_mannual_pay,if_change_card,if_change_processor,direction_change_processor,monthly_vintage
index,,,,,,,,,,,,,,,,,,,,
1,327,-1,67.784465,23,Fri,Stripe,10.0,DEBIT,USAA,1757.36,2987.57,11,632.0,6.0,fortress1,1,1,0,NaN,2018-10
2,327,-1,67.784465,23,Fri,Stripe,0.0,DEBIT,USAA SAVINGS BANK,190.50,1882.19,11,717.0,4.0,fortress1,1,1,0,NaN,2018-6
3,327,-1,67.784465,23,Fri,Stripe,NaN,DEBIT,BMO HARRIS BANK N.A.,270.51,322.88,11,718.0,1.0,fortress1,1,0,0,NaN,2019-1


In [44]:
import pandas as pd
import sklearn as sk
import numpy as np
from vowpalwabbit import pyvw

print(len(df_merged.arms.unique())) # 502 unique arms
vw = pyvw.vw("--cb 502")
# vw = pyvw.vw("--cb_explore 502 -q UA --quiet --epsilon 0.2")
# vw = pyvw.vw("--cb_explore_adf -q UA --quiet --epsilon 0.2")


502


### Train/Learn

In [45]:
# Just testing last 20000 rows for vowpal wabbit
train_df = df_merged[-20000:]

for i in train_df.index:
    action = train_df.loc[i, "arms"]
    cost = train_df.loc[i, "reward"]
    probability = train_df.loc[i, "probability"]
    feature1 = train_df.loc[i, "hour"]
    feature2 = train_df.loc[i, "dayofweek"]
    feature3 = train_df.loc[i, "transaction_provider"]
    feature4 = train_df.loc[i, "txn_duration"]
    feature5 = train_df.loc[i, "type"]
    feature6 = train_df.loc[i, "bank"]
    feature7 = train_df.loc[i, "transaction_amount"]
    feature8 = train_df.loc[i, "financed_amount"]
    feature9 = train_df.loc[i, "term"]
    feature10 = train_df.loc[i, "fico"]
    feature11 = train_df.loc[i, "payment_index"]
    feature12 = train_df.loc[i, "destination_account"]
    feature13 = train_df.loc[i, "if_mannual_pay"]
    feature14 = train_df.loc[i, "if_change_card"]
    feature15 = train_df.loc[i, "if_change_processor"]
    feature16 = train_df.loc[i, "direction_change_processor"]
    feature17 = train_df.loc[i, "monthly_vintage"]

    # Construct the example in the required vw format.
    learn_example = str(action) + ":" + str(cost) + ":" + str(probability) + " | " + \
                    str(feature1) + " " + str(feature2) + " " + str(feature3) + " " + \
                    str(feature4) + " " + str(feature5) + " " + str(feature6) + " " + \
                    str(feature7) + " " + str(feature8) + " " + str(feature9) + " " + \
                    str(feature10) + " " + str(feature11) + " " + str(feature12) + " " + \
                    str(feature13) + " " + str(feature14) + " " + str(feature15) + " " + \
                    str(feature16) + " " + str(feature17)
                

    # Here we do the actual learning.
    vw.learn(learn_example)

In [46]:
# Test on last 20 records
exclude = ['arms', 'reward', 'probability']
test_df = df_merged[ [ col for col in df_merged.columns if col not in exclude ] ]
test_df = test_df[-20:].reset_index(drop=True)
test_df['index'] = range(1, len(test_df) + 1)
test_df = test_df.set_index("index")
test_df.head()

,hour,dayofweek,transaction_provider,txn_duration,type,bank,transaction_amount,financed_amount,term,fico,payment_index,destination_account,if_mannual_pay,if_change_card,if_change_processor,direction_change_processor,monthly_vintage
index,,,,,,,,,,,,,,,,,
1,12,Thu,Chase,7.0,DEBIT,RBS CITIZENS N.A.,303.00,2945.06,11,700.0,13.0,creditsuisse,1,0,0,NaN,2019-12
2,12,Thu,Chase,8.0,DEBIT,WELLS FARGO BANK N.A.,57.07,528.93,11,646.0,9.0,creditsuisse,1,0,0,NaN,2019-12
3,10,Sun,Chase,2.0,DEBIT,BANK OF AMERICA N.A.,128.00,1292.17,11,670.0,11.0,creditsuisse,1,0,0,NaN,2019-11
4,10,Sun,Chase,24.0,DEBIT,BANK OF AMERICA N.A.,80.00,550.84,11,655.0,8.0,creditsuisse,1,0,0,NaN,2020-3
5,17,Sun,Chase,3.0,DEBIT,FIFTH THIRD BANK THE,524.34,5051.19,11,684.0,4.0,creditsuisse,0,0,0,NaN,2020-6


In [47]:
for i in test_df.index:
    feature1 = test_df.loc[i, "hour"]
    feature2 = test_df.loc[i, "dayofweek"]
    feature3 = test_df.loc[i, "transaction_provider"]
    feature4 = test_df.loc[i, "txn_duration"]
    feature5 = test_df.loc[i, "type"]
    feature6 = test_df.loc[i, "bank"]
    feature7 = test_df.loc[i, "transaction_amount"]
    feature8 = test_df.loc[i, "financed_amount"]
    feature9 = test_df.loc[i, "term"]
    feature10 = test_df.loc[i, "fico"]
    feature11 = test_df.loc[i, "payment_index"]
    feature12 = test_df.loc[i, "destination_account"]
    feature13 = test_df.loc[i, "if_mannual_pay"]
    feature14 = test_df.loc[i, "if_change_card"]
    feature15 = test_df.loc[i, "if_change_processor"]
    feature16 = test_df.loc[i, "direction_change_processor"] # Maybe just previous processor?
    feature17 = test_df.loc[i, "monthly_vintage"]

    test_example = "| " + str(feature1) + " " + str(feature2) + " " + str(feature3) + " " + \
                    str(feature4) + " " + str(feature5) + " " + str(feature6) + " " + \
                    str(feature7) + " " + str(feature8) + " " + str(feature9) + " " + \
                    str(feature10) + " " + str(feature11) + " " + str(feature12) + " " + \
                    str(feature13) + " " + str(feature14) + " " + str(feature15) + " " + \
                    str(feature16) + " " + str(feature17)

    choice = vw.predict(test_example)
    print(i, choice)

1 135
2 156
3 71
4 127
5 143
6 162
7 159
8 65
9 135
10 61
11 69
12 104
13 104
14 111
15 473
16 145
17 143
18 134
19 132
20 104


In [48]:
# Save CMAB
vw.save('cb.model')
del vw

# Load CMAB for more predictions
# vw = pyvw.vw("--cb 4 -i cb.model")
# print(vw.predict('| a b'))

In [ ]:
# Step 1; split retries & sanity check.
# Take transactions we ran today. See what it suggests vs. what we did.
# Split on first and retries.
# Stripe/Wells Fargo same MCC code. Chase different.

# Step 2; Optimize Reward/Cost Function.
# Iteration; Brianna.
# Durbin Amendment vs. Non for Reward change
# MCC code bases the risk profile from bank. Risker is higher MCC code and each transaction is 2.5% for risky businesses..

# Step 3; Carlos.
# Exact cost structure per/transaction.

In [2]:
# train_data = [{'action': 1, 'cost': 2, 'probability': 0.4, 'feature1': 'a', 'feature2': 'c', 'feature3': ''},
#               {'action': 3, 'cost': 0, 'probability': 0.2, 'feature1': 'b', 'feature2': 'd', 'feature3': ''},
#               {'action': 4, 'cost': 1, 'probability': 0.5, 'feature1': 'a', 'feature2': 'b', 'feature3': ''},
#               {'action': 2, 'cost': 1, 'probability': 0.3, 'feature1': 'a', 'feature2': 'b', 'feature3': 'c'},
#               {'action': 3, 'cost': 1, 'probability': 0.7, 'feature1': 'a', 'feature2': 'd', 'feature3': ''}]

# train_df = pd.DataFrame(train_data)

# # Add index to data frame
# train_df['index'] = range(1, len(train_df) + 1)
# train_df = train_df.set_index("index")

# from vowpalwabbit import pyvw

# vw = pyvw.vw("--cb 4")

# for i in train_df.index:
#     action = train_df.loc[i, "action"]
#     cost = train_df.loc[i, "cost"]
#     probability = train_df.loc[i, "probability"]
#     feature1 = train_df.loc[i, "feature1"]
#     feature2 = train_df.loc[i, "feature2"]
#     feature3 = train_df.loc[i, "feature3"]

#     # Construct the example in the required vw format.
#     learn_example = str(action) + ":" + str(cost) + ":" + str(probability) + " | " + str(feature1) + " " + str(feature2) + " " + str(feature3)

#     # Here we do the actual learning.
#     vw.learn(learn_example)

# test_data = [{'feature1': 'b', 'feature2': 'c', 'feature3': ''},
#             {'feature1': 'a', 'feature2': '', 'feature3': 'b'},
#             {'feature1': 'b', 'feature2': 'b', 'feature3': ''},
#             {'feature1': 'a', 'feature2': '', 'feature3': 'b'}]

# test_df = pd.DataFrame(test_data)

# # Add index to data frame
# test_df['index'] = range(1, len(test_df) + 1)
# test_df = test_df.set_index("index")

# for j in test_df.index:
#     feature1 = test_df.loc[j, "feature1"]
#     feature2 = test_df.loc[j, "feature2"]
#     feature3 = test_df.loc[j, "feature3"]

#     test_example = "| " + str(feature1) + " " + str(feature2) + " " + str(feature3)

#     choice = vw.predict(test_example)
#     print(j, choice)